In [5]:
#最初版本 未更寫IDW前

In [6]:
import json
import math
import fiona
import folium
import branca.colormap as cm
import requests
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import urllib.request
from urllib import request
from shapely.geometry import shape, Point
import os
import threading
from rtree import index
import time
import datetime
import gzip
from pandas.io.json import json_normalize

In [60]:
#idw functuion

def idw(lat , lon , ref_point):
    total_out = 0
    total_dis = 0
    # optional normalize
    size = 0.01
    sort_list = pd.DataFrame(columns=['Latitude','Longitude','PM2.5','distance'])
    # sort distance
    sort_list['Latitude']=(((ref_point['Latitude']-lat)/size)**2)
    sort_list['Longitude']=(((ref_point['Longitude']-lon)/size)**2)
    sort_list['PM2.5']=(ref_point['PM2.5'])
    sort_list['distance']=np.sqrt(sort_list['Latitude']+sort_list['Longitude'])
    sort_list=sort_list.sort_values(by=['distance'])
    sort_list=sort_list.reset_index()
    
    if sort_list.loc[0,'distance']==0:
        return sort_list.loc[0,'PM2.5']
    
    sort_list['distance']=(1/sort_list['distance'])
    total_dis=sum(sort_list.loc[0:3,'distance'])
    total_out=sum((sort_list.loc[0:3,'distance']/total_dis)*sort_list.loc[0:3,'PM2.5'])
    return total_out # 預估的pm2.5

In [61]:
'''#空氣盒子


gzfileUrl="https://pm25.lass-net.org/data/last-all-airbox.json.gz"
gzflieSaveName="last-all-airbox.json.gz"
r = requests.get(gzfileUrl)
open(gzflieSaveName, 'wb').write(r.content)

f = gzip.open(gzflieSaveName,'r')
jdata = f.read().decode("utf-8")
f.close()
data = json.loads(jdata)
airbox = pd.DataFrame.from_dict(data['feeds']);
#airbox = airbox.loc[airbox["device_name"].str.contains('台中|臺中')]

airbox['areaname']=''
airbox.index= range(1,len(airbox) + 1)
airbox['Id']=airbox.index
airbox.index= range(0,len(airbox))
#times_airbox = airbox['datetime']
airbox
'''

'#空氣盒子\n\n\ngzfileUrl="https://pm25.lass-net.org/data/last-all-airbox.json.gz"\ngzflieSaveName="last-all-airbox.json.gz"\nr = requests.get(gzfileUrl)\nopen(gzflieSaveName, \'wb\').write(r.content)\n\nf = gzip.open(gzflieSaveName,\'r\')\njdata = f.read().decode("utf-8")\nf.close()\ndata = json.loads(jdata)\nairbox = pd.DataFrame.from_dict(data[\'feeds\']);\n#airbox = airbox.loc[airbox["device_name"].str.contains(\'台中|臺中\')]\n\nairbox[\'areaname\']=\'\'\nairbox.index= range(1,len(airbox) + 1)\nairbox[\'Id\']=airbox.index\nairbox.index= range(0,len(airbox))\n#times_airbox = airbox[\'datetime\']\nairbox\n'

In [62]:
#========================================================================================================
#爬蟲 (環保署測站、時間、風力資訊、台中各區天氣) + idw

ses = requests.Session()
data1 = ses.get('http://taqm.epb.taichung.gov.tw/TQAMNEWAQITABLE.ASPX') #環保署16筆測站
data1.encoding = 'utf-8'
t = pd.read_html(data1.text)[0]
t.drop(t.iloc[:, 1:21], inplace=True, axis=0)
t.drop(t.iloc[:, 1:279], inplace=True, axis=1)
times = str(t[0])



In [63]:
#爬蟲 微型感測器

url = 'https://aqi.thu.edu.tw/echarts/getEPAIotData'
json_data = request.urlopen(url).read().decode("utf-8")
json_data = json.loads(json_data);
frame = pd.DataFrame.from_dict(json_data);

frame['areaname']=''
frame.index= range(1,len(frame) + 1)
frame['Id']=frame.index
frame.index= range(0,len(frame))
times_micro = frame['datetime']
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-20 00:24:13,1858224692,24.975056,121.330402,7,,1
1,2019-12-20 00:28:36,1858300612,24.956991,121.345565,-99,,2
2,2019-12-20 00:27:54,1858495000,25.042832,121.137675,-99,,3
3,2019-12-20 00:11:56,1858569336,24.967298,121.339279,7,,4
4,2019-12-20 00:26:31,1858763627,24.973724,121.332481,7,,5
5,2019-12-20 00:28:44,1858852981,24.972902,121.342242,7,,6
6,2019-12-19 16:37:00,1859886331,24.965728,121.332249,7,,7
7,2019-12-20 00:24:01,1860067717,24.94805,121.341753,7,,8
8,2019-12-20 00:27:25,1860271203,24.956835,121.360893,7,,9
9,2019-12-20 00:28:12,1860464577,24.973736,121.328505,8,,10


In [64]:
#找出位於台中的微型感測器
#先找出微型感測器所在位置
def getareaname(lng,lat,num):
   
    lng = float(lng)
    lat = float(lat)
    
    area_append['areaname'][num]=search(float(lng), float(lat))
    return 0

def get_areanamebythread(totaldata):
    totalnum=totaldata  #總執行次數
    Q=int(totalnum/5) #取商數
    R=totalnum%5     #取餘數
    
    for i in range(Q):
        threads = []
        for j in range(5):#開多少執行緒
            threads.append(threading.Thread(
                target=getareaname,#要執行函數
                args=(frame['longitude'][i*5+j],frame['latitude'][i*5+j],i*5+j)))#要執行函數的參數
            threads[j].start()
        for j in threads:
            j.join()
        print(round(float((i+1)*100/totalnum*5),2),'%')#顯示進度
    
    threads = []
    for i in range(R):
        threads.append(threading.Thread(
            target=getareaname,
            args=(frame['longitude'][Q*5+i],frame['latitude'][Q*5+i],Q*5+i)))
        threads[i].start()
    for j in threads:
        j.join()
    print("100%")#顯示進度

def search(x, y):  #尋找鄉鎮
    global shapes, townnames
    #idx = index.Index()
    #for town_id, shape in shapes.items():
    #idx.insert(town_id,shape.bounds)
    return next((townnames[town_id]  #如果鄉鎮區域包含傳入的經緯度就傳回townnames[town_id]
                 for town_id in shapes #idx.intersection((x,y)) #逐一尋找各鄉鎮
                 if shapes[town_id].contains(Point(x, y))), None)

area_append = pd.DataFrame(columns=['areaname'])
area_append['areaname'] = frame['datetime']



module_dir = os.path.dirname('/home/gh555657/123321/areasearch/')  #取得目前目錄
collection = fiona.open(os.path.join(module_dir,'TOWN_MOI_1070516.shp'))
shapes = {}
townnames = {}

for f in collection:
    town_id =f['properties']['TOWNCODE'] #鄉鎮代碼
    shapes[town_id] = shape(f['geometry'])  #鄉鎮界限經緯度
    townnames[town_id] = f['properties']['COUNTYNAME'] + f['properties']['TOWNNAME']#search函式傳回值
    
t1=time.time()
get_areanamebythread(frame.shape[0])
t2=time.time()
print('gevent-time:%s' % str(t2-t1))
frame['areaname']=area_append['areaname']


0.18 %
0.35 %
0.53 %
0.7 %
0.88 %
1.06 %
1.23 %
1.41 %
1.58 %
1.76 %
1.94 %
2.11 %
2.29 %
2.46 %
2.64 %
2.81 %
2.99 %
3.17 %
3.34 %
3.52 %
3.69 %
3.87 %
4.05 %
4.22 %
4.4 %
4.57 %
4.75 %
4.93 %
5.1 %
5.28 %
5.45 %
5.63 %
5.81 %
5.98 %
6.16 %
6.33 %
6.51 %
6.69 %
6.86 %
7.04 %
7.21 %
7.39 %
7.57 %
7.74 %
7.92 %
8.09 %
8.27 %
8.44 %
8.62 %
8.8 %
8.97 %
9.15 %
9.32 %
9.5 %
9.68 %
9.85 %
10.03 %
10.2 %
10.38 %
10.56 %
10.73 %
10.91 %
11.08 %
11.26 %
11.44 %
11.61 %
11.79 %
11.96 %
12.14 %
12.32 %
12.49 %
12.67 %
12.84 %
13.02 %
13.19 %
13.37 %
13.55 %
13.72 %
13.9 %
14.07 %
14.25 %
14.43 %
14.6 %
14.78 %
14.95 %
15.13 %
15.31 %
15.48 %
15.66 %
15.83 %
16.01 %
16.19 %
16.36 %
16.54 %
16.71 %
16.89 %
17.07 %
17.24 %
17.42 %
17.59 %
17.77 %
17.95 %
18.12 %
18.3 %
18.47 %
18.65 %
18.82 %
19.0 %
19.18 %
19.35 %
19.53 %
19.7 %
19.88 %
20.06 %
20.23 %
20.41 %
20.58 %
20.76 %
20.94 %
21.11 %
21.29 %
21.46 %
21.64 %
21.82 %
21.99 %
22.17 %
22.34 %
22.52 %
22.7 %
22.87 %
23.05 %
23.22 %
23.4 %
23.57

In [65]:
frame['pm25'].replace({"NA":np.nan}, inplace=True)
frame['pm25'].astype('float64')
frame['pm25'].replace({"-99":np.nan,"-99.0":np.nan}, inplace=True)
frame.dropna(inplace=True)
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-20 00:24:13,1858224692,24.975056,121.330402,7.0,新北市鶯歌區,1
3,2019-12-20 00:11:56,1858569336,24.967298,121.339279,7.0,新北市鶯歌區,4
4,2019-12-20 00:26:31,1858763627,24.973724,121.332481,7.0,新北市鶯歌區,5
5,2019-12-20 00:28:44,1858852981,24.972902,121.342242,7.0,新北市鶯歌區,6
6,2019-12-19 16:37:00,1859886331,24.965728,121.332249,7.0,新北市鶯歌區,7
7,2019-12-20 00:24:01,1860067717,24.94805,121.341753,7.0,新北市鶯歌區,8
8,2019-12-20 00:27:25,1860271203,24.956835,121.360893,7.0,新北市鶯歌區,9
9,2019-12-20 00:28:12,1860464577,24.973736,121.328505,8.0,新北市鶯歌區,10
10,2019-12-20 00:26:37,1860726455,24.95694,121.358481,7.0,新北市鶯歌區,11
12,2019-12-19 18:01:25,1861018038,24.950129,121.348118,7.0,新北市鶯歌區,13


In [66]:
now_time=(datetime.datetime.now()+datetime.timedelta(hours=0)).strftime("%Y-%m-%d %H")
frame=frame.loc[frame["datetime"].str.contains(now_time)]
frame.to_csv('/home/gh555657/123321/taiwan_all_micro.csv', mode='a', header=False)

In [67]:
frame

,datetime,device_id,latitude,longitude,pm25,areaname,Id
0,2019-12-20 00:24:13,1858224692,24.975056,121.330402,7.0,新北市鶯歌區,1
3,2019-12-20 00:11:56,1858569336,24.967298,121.339279,7.0,新北市鶯歌區,4
4,2019-12-20 00:26:31,1858763627,24.973724,121.332481,7.0,新北市鶯歌區,5
5,2019-12-20 00:28:44,1858852981,24.972902,121.342242,7.0,新北市鶯歌區,6
7,2019-12-20 00:24:01,1860067717,24.94805,121.341753,7.0,新北市鶯歌區,8
8,2019-12-20 00:27:25,1860271203,24.956835,121.360893,7.0,新北市鶯歌區,9
9,2019-12-20 00:28:12,1860464577,24.973736,121.328505,8.0,新北市鶯歌區,10
10,2019-12-20 00:26:37,1860726455,24.95694,121.358481,7.0,新北市鶯歌區,11
14,2019-12-20 00:27:54,1862163982,24.956971,121.353984,7.0,新北市鶯歌區,15
15,2019-12-20 00:26:51,1862457890,24.970894,121.345517,8.0,新北市鶯歌區,16


In [68]:
epa = frame.loc[frame["areaname"].str.contains('台中|臺中')]
epa=epa.drop(columns=['datetime','areaname']).reset_index(drop=True)
epa.columns=['SiteName','Latitude','Longitude','PM2.5','Id']
epa.index= range(1,len(epa) + 1)
epa['Id']=epa.index
epa.index= range(0,len(epa))
epaidw=epa
epa=epa[['SiteName','Id','PM2.5','Latitude','Longitude']]
epaidw=epaidw.drop(columns=['SiteName']).reset_index(drop=True)
epa

,SiteName,Id,PM2.5,Latitude,Longitude
0,6170369574,1,7.0,24.161848,120.6049575
1,6170479504,2,4.0,24.237501,120.517601
2,6170511673,3,3.0,24.241976,120.523674
3,6170644194,4,5.0,24.15854,120.6066032
4,6170777118,5,5.0,24.166212,120.5971678
5,6170852684,6,10.0,24.206862,120.5050963
6,6170988978,7,4.0,24.234808,120.5295139
7,6171173697,8,4.0,24.236878,120.5057026
8,6171278726,9,5.0,24.242703,120.5328221
9,6171387123,10,4.0,24.303089,120.7221755


In [69]:
epaidw.columns=['Latitude','Longitude','PM2.5','Id']

In [70]:

epaidw=epaidw[['Id','Latitude','Longitude','PM2.5']]



In [71]:
# print(times[21:36]) 時間
df1 = pd.read_html(data1.text)[1]
cols1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df1 = df1.replace('─','0')
df1 = df1.drop(df1.columns[cols1],axis=1)
df1.rename(columns={ df1.columns[0]: "SiteName"}, inplace=True)
df1.rename(columns={ df1.columns[1]: "PM2.5"}, inplace=True)
df1['Latitude']=[24.1622,24.151958,24.099611,24.225628,
                 24.256586,24.139008,24.350426,24.139564,
                 24.05735,24.094264,24.307036,24.250388,
                 24.150919,24.182055,24.269233,24.20103]

df1['Longitude']=[120.616917,120.641092,120.677689,120.568794,
                  120.741711,120.597876,120.615358,120.715064,
                  120.697299,120.646629,120.714881,120.538839,
                  120.540877,120.60707,120.576421,120.498566]


In [72]:
df1.rename(columns={"測站名稱":"SiteName"}, inplace=True)
df1.rename(columns={ "細懸浮微粒(PM2.5)":"PM2.5"}, inplace=True)


df1

,SiteName,PM2.5,Latitude,Longitude
,SiteName,移動平均濃度(μg/m3),,
0,西屯測站(環保署),9,24.162200,120.616917
1,忠明測站(環保署),12,24.151958,120.641092
2,大里測站(環保署),12,24.099611,120.677689
3,沙鹿測站(環保署),8,24.225628,120.568794
4,豐原測站(環保署),10,24.256586,120.741711
5,文山測站,16,24.139008,120.597876
6,大甲測站,7,24.350426,120.615358
7,太平測站,17,24.139564,120.715064
8,霧峰測站,18,24.057350,120.697299


In [73]:
df1[['Latitude', 'Longitude', 'PM2.5']] = df1[['Latitude', 'Longitude','PM2.5']].astype(float)
#df1  顯示16筆測站
df1.to_csv("/home/gh555657/123321/df1.csv")
df1 = pd.read_csv("/home/gh555657/123321/df1.csv")



In [74]:

data = ses.get('https://www.cwb.gov.tw/V7/observe/real/ObsC.htm?')
data.encoding = 'utf-8'
wind = pd.read_html(data.text)[0]
wind.drop(wind.iloc[:, 2:4], inplace=True, axis=1)
wind.drop(wind.iloc[:, 4:11], inplace=True, axis=1)
wind = wind.drop([30])
wind.drop('日照時數',inplace=True,axis=1)
wind.drop(wind.index[:2],inplace=True)
wind.rename(columns={ wind.columns[0]: "Sitename" }, inplace=True)
wind.rename(columns={ wind.columns[1]: "time" }, inplace=True)
wind.rename(columns={ wind.columns[2]: "Wind direction" }, inplace=True)
wind.rename(columns={ wind.columns[3]: "Wind speed" }, inplace=True)

wind['Latitude']=[24.256002,24.15295,24.388594,24.276061,24.248517,24.381675,
                  24.363714,24.246428,24.247522,24.347667,24.173142,24.103556,
                  24.272481,24.3593912,24.310436,24.254322,24.092464,24.21310,
                  24.312297,24.347817,24.184536,24.107058,24.179494,24.137006,
                  24.200172,24.215281,24.432739,24.388792,24.322831]
wind['Longitude']=[120.5233806,120.572117,121.236339,120.777644,120.903319,121.4203,
                   121.444667,120.833047,121.243669,120.640403,120.722289,120.751061,
                   120.658314,120.5849602,120.729725,120.720692,120.701378,120.703933,
                   120.562242,120.705686,120.528972,120.624103,120.641275,120.637969,
                   120.815806,120.62445,121.303808,121.268675,120.682686]
wind.replace('-', 0,inplace=True)
wind.replace('－', '無資料',inplace=True)
wind = wind.reset_index()
#wind 顯示風力資訊
#wind.to_csv("C:\\final_project\\final_project_epa\\wind.csv",encoding="utf_8_sig")
#wind = pd.read_csv("C:\\final_project\\final_project_epa\\wind.csv",encoding="utf_8_sig")



In [77]:
df1.drop([0],inplace=True) #把df1不要的第一row砍掉
df11=df1['SiteName']       #先測站把名稱存進df11
df11=df11.append(epa['SiteName'])#在把微型感測器測站名稱也併進df11

df1.drop(["SiteName"],axis=1,inplace=True)#測站名稱column砍掉
df1.rename(columns={ 'Unnamed: 0':'Id'}, inplace=True)#更改unname--->Id
#df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']] df1目前型態
#df1=df1.append(epaidw) #把微型感測器資料也併進來
df1=df1.astype('float64')#轉成float64
df11.index= range(0,len(df11))#index重排
df1.index=range(0,len(df1))

In [78]:
#taichung = gp.read_file("/home/hpc/taichungcity.geojson")           #台中邊界
taichungmap_1x1 = gp.read_file("/home/gh555657/123321/final.geojson")         #台中1*1網格
taichung_district = gp.read_file("/home/gh555657/123321/taichung_district.geojson")
#list1= [   1,    4,   14,   26,   44,   63,   82,  102,  122,  144,
#         168,  193,  221,  257,  304,  353,  403,  455,  510,  568,
#         627,  687,  750,  819,  892,  968, 1053, 1141, 1232, 1325,
#        1418, 1510, 1601, 1692, 1781, 1864, 1944, 2019, 2087, 2145,
#        2197, 2246, 2289, 2330, 2359, 2384, 2403, 2419, 2433, 2445 ]
#list2= [   3,   13,   25,   43,   62,   81,  101,  121,  143,  167,
#         192,  220,  256,  303,  352,  402,  454,  509,  567,  626,
#         686,  749,  818,  891,  967, 1052, 1140, 1231, 1324, 1417,
#        1509, 1600, 1691, 1780, 1863, 1943, 2018, 2086, 2144, 2196,
#        2245, 2288, 2329, 2358, 2383, 2402, 2418, 2432, 2444, 2449 ]
lon_max=taichungmap_1x1.bounds.maxx
lon_min=taichungmap_1x1.bounds.minx
lat_max=taichungmap_1x1.bounds.maxy
lat_min=taichungmap_1x1.bounds.miny



In [79]:
#idw=====================================================
# 重要參數歸零
total_out = 0
total_dis = 0
# optional normalize，
size = 0.0092
sort_list = pd.DataFrame([])
sort_list['Latitude']   = df1['Latitude']
sort_list['Longitude']  = df1['Longitude']
sort_list['PM2.5']      = df1['PM2.5']
sort_list['Lat_dis_km'] = abs((df1['Latitude'] - 23.999907) /size)
sort_list['Lon_dis_km'] = abs((df1['Longitude']-120.739079) /size)
sort_list['distance']=np.sqrt(sort_list['Lat_dis_km']**2+sort_list['Lon_dis_km']**2)

sort_list=sort_list.sort_values(by=['distance'])
sort_list=sort_list.reset_index()

print(sort_list.shape)
sort_list.head(40)
sort_list.loc[0,'distance']

total_out_center = 0
total_out_L1 = 0
total_out_L2 = 0

for d in range(5):
    center_limit = 5
    if d*5<=center_limit:
        center_list = sort_list.loc[(sort_list['distance']<d*5)]
        total_out_center = (center_list['PM2.5']).mean()    


total_out_center

# df3 idw point

df3 = pd.DataFrame(columns=['Latitude','Longitude','PM2.5','Id'])

site_name_count = 1

for i, j, k, l in zip(lat_max, lat_min, lon_max, lon_min):

    site_name = str(site_name_count)
    site_name_count += 1

    lat_mean = (i+j)/2
    lon_mean = (k+l)/2

# idw_value，指的是PM2.5估計值，共需要估計2449個網格。
    idw_value = idw(lat_mean,lon_mean,df1)

# 2449個網格的中心點由2449個(lat_mean,lon_mean)組成，命名為流水號(1~2449)。
    df_append_t1 = [[lat_mean,lon_mean,idw_value,site_name]]
    df_append = pd.DataFrame(df_append_t1,columns=['Latitude', 'Longitude', 'PM2.5', 'Id'])

    df3 = df3.append(df_append)        #合併
#=========================================================
df3.to_csv("/home/gh555657/123321/all_point_data_epa.csv")
df3.to_csv("/home/gh555657/123321/testidw_original.csv")
all_point_data_epa = pd.read_csv("/home/gh555657/123321/all_point_data_epa.csv")
lon=list(all_point_data_epa['Longitude'])
lat=list(all_point_data_epa['Latitude'])
#all_point_data_epa['Id']=0
Id=taichungmap_1x1['Id']
ans_Id=all_point_data_epa['Id']


(16, 7)


In [80]:
def generalID(lon,lat,column_num,row_num):
    # 若在范围外的点，返回-1
    if lon <= LON1 or lon >= LON2 or lat <= LAT1 or lat >= LAT2:
        return -1
    # 把经度范围根据列数等分切割
    column = (LON2 - LON1)/column_num
    # 把纬度范围根据行数数等分切割
    row = (LAT2 - LAT1)/row_num
    # 二维矩阵坐标索引转换为一维ID，即： （列坐标区域（向下取整）+ 1） + （行坐标区域 * 列数）
    return int((lon-LON1)/column)+ 1 + int((lat-LAT1)/row) * column_num


In [81]:
ans_Id

0          1
1          2
2          3
3          4
4          5
5          6
6          7
7          8
8          9
9         10
10        11
11        12
12        13
13        14
14        15
15        16
16        17
17        18
18        19
19        20
20        21
21        22
22        23
23        24
24        25
25        26
26        27
27        28
28        29
29        30
        ... 
2419    2420
2420    2421
2421    2422
2422    2423
2423    2424
2424    2425
2425    2426
2426    2427
2427    2428
2428    2429
2429    2430
2430    2431
2431    2432
2432    2433
2433    2434
2434    2435
2435    2436
2436    2437
2437    2438
2438    2439
2439    2440
2440    2441
2441    2442
2442    2443
2443    2444
2444    2445
2445    2446
2446    2447
2447    2448
2448    2449
Name: Id, Length: 2449, dtype: int64

In [82]:
# fit id
#for i in range(2449):
#    ans_Id[i] = i+1
# for d in (range(100)):      #測試資料筆數
#      for m in (range(50)):  #找是在哪列
#             if lon_min[list1[m]-1]<lon[d] and lon[d]<lon_max[list2[m]-1] and lat_min[list1[m]-1]<lat[d] and lat[d]<lat_max[list2[m]-1]:
#                 for n in range(list1[m],list2[m]+1): #找是在該列的哪個
#                     if lon_min[n-1]<lon[d] and lon[d]<lon_max[n-1] and lat_min[n-1]<lat[d] and lat[d]<lat_max[n-1]:
#                         ans_Id[d]=Id[n]-1

#taichung_district

In [83]:
taichungmap_1x1 = taichungmap_1x1.merge(all_point_data_epa, on='Id')
taichungmap_1x1['PM2.5']=taichungmap_1x1['PM2.5'].round()
taichung_district = taichung_district.merge(weather, on='區名')


In [84]:
df1['SiteName']=df11
df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']]


In [85]:
df3

,Latitude,Longitude,PM2.5,Id
0,23.999907,120.739079,16.028233,1
0,23.999923,120.748907,16.006071,2
0,23.999939,120.758736,15.984507,3
0,24.008824,120.680084,16.044917,4
0,24.008844,120.689914,16.093393,5
0,24.008864,120.699743,16.122090,6
0,24.008883,120.709572,16.127521,7
0,24.008901,120.719402,16.113373,8
0,24.008919,120.729231,16.087293,9
0,24.008936,120.739061,16.056585,10


In [88]:
# =============================================================================================================
# folium

variable  = 'PM2.5'
colorList = ['#98fb98','#00ff00','#32cd32','#ffff00','#ffd700','#ffa500','#ff6347','#ff0000','#ba55d3']
map_color = cm.StepColormap(colorList,index=[0,10,20,30,40,50,60,70,80],vmin=0,vmax=100,caption = 'PM2.5')

fmap = folium.Map(location=[24.2, 120.9], zoom_start=10.5)
# fmap.choropleth(
#                geo_data=taichungmap_1x1,
#                name='pm2.5',
#                columns=['Id', 'PM2.5'],
#                key_on='feature.properties.Id',
#                data=all_point_data_epa,
#               #threshold_scale=[],
#                fill_color='BuGn',
#                legend_name='pm2.5',
#                line_opacity=0.5,
#                fill_opacity=0.8
#                )
folium.GeoJson(taichungmap_1x1,
               name='PM2.5',
               style_function=lambda x: {
                   'fillColor': map_color(x['properties'][variable]),
                   'color': 'black',
                   'weight': 0.5,
                   'fillOpacity': 0.7
               },
               highlight_function=lambda x: {
                   'weight': 3,
                   'color': 'black'
               },
               tooltip=folium.GeoJsonTooltip(fields=['Id', 'PM2.5'],
                                             aliases=['Id', 'PM2.5'],
                                             labels=True,
                                             sticky=True)).add_to(fmap)
folium.GeoJson(taichung_district,
               name='台中各區天氣',
               style_function=lambda x: {
                   "weight": 1,
                   'color': '#5499c7'
               },
               highlight_function=lambda x: {
                   'weight': 3,
                   'color': '#2471A3'
               },
               tooltip=folium.GeoJsonTooltip(
                   fields=['區名', '天氣', '溫度', '降雨預報'],
                   aliases=['區名', '天氣', '溫度', '降雨機率'],
                   labels=True,
                   sticky=False),
               show=False).add_to(fmap)
# fmap.choropleth(
#               geo_data=taichung,
#               name='taichung',
#               line_opacity=0.5,
#               fill_opacity=0
#                )
#微型感測器logo
epa_micro_url= 'https://ci.taiwan.gov.tw/dsp/img/map_icon/air_quality.png'
# 環保署 logo
epa_icon_url = 'https://www.epa.gov.tw/public/MMO/epa/Epa_Logo_01_LOGO.gif'
# 風力圖標 logo (十六方位)
wind_icon_url = [
    'https://www.cwb.gov.tw/V7/images/wind_icon/N.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/NNE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/NE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/ENE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/E.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/ESE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/SE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/SSE.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/S.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/SSW.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/SW.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/WSW.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/W.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/WNW.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/NW.gif',
    'https://www.cwb.gov.tw/V7/images/wind_icon/NNW.gif'
]

station = folium.FeatureGroup(name="環保署", show=True)
for i in (range(15)):
    station.add_child(
        folium.Marker(
            location=[df1['Latitude'][i], df1['Longitude'][i]],
            popup=("<b>NAME:</b> {NAME}<br>"
                   " <p><b>PM2.5:</b> {PM25}<br>"
                   " <p><b>TIME:</b> {TIME}<br>").format(
                       NAME=str(df1['SiteName'][i]),
                       PM25=str(df1['PM2.5'][i]),
                       TIME=str(times[39:55])),
            icon=folium.CustomIcon(epa_icon_url,
                                   icon_size=(23,
                                              23))  # Creating a custom Icon
        ))


station2 = folium.FeatureGroup(name="風力", show=False)
for j in (range(29)):
    if wind['Wind direction'][j] == '北':
        x = 0
    elif wind['Wind direction'][j] == '北北東':
        x = 1
    elif wind['Wind direction'][j] == '東北':
        x = 2
    elif wind['Wind direction'][j] == '東北東':
        x = 3
    elif wind['Wind direction'][j] == '東':
        x = 4
    elif wind['Wind direction'][j] == '東南東':
        x = 5
    elif wind['Wind direction'][j] == '東南':
        x = 6
    elif wind['Wind direction'][j] == '南南東':
        x = 7
    elif wind['Wind direction'][j] == '南':
        x = 8
    elif wind['Wind direction'][j] == '南南西':
        x = 9
    elif wind['Wind direction'][j] == '西南':
        x = 10
    elif wind['Wind direction'][j] == '西南西':
        x = 11
    elif wind['Wind direction'][j] == '西':
        x = 12
    elif wind['Wind direction'][j] == '西北西':
        x = 13
    elif wind['Wind direction'][j] == '西北':
        x = 14
    elif wind['Wind direction'][j] == '北北西':
        x = 15
    elif wind['Wind direction'][j] == '靜風':
        x = 0

    station2.add_child(
        folium.Marker(location=[wind['Latitude'][j], wind['Longitude'][j]],
                      popup=("<b>NAME:</b> {NAME}<br>"
                             "<b>Wind speed:</b> {Windspeed}<br>"
                             "<b>Wind direction:</b> {winddirection}<br>"
                             "<b>TIME:</b> {TIME}<br>").format(
                                 NAME=str(wind['Sitename'][j]),
                                 Windspeed=str(wind['Wind speed'][j]),
                                 winddirection=str(wind['Wind direction'][j]),
                                 TIME=str(times[39:55])),
                      icon=folium.CustomIcon(wind_icon_url[x],
                                             icon_size=(27, 27))))

fmap.add_child(station)
fmap.add_child(station2)

fmap.add_child(map_color)
folium.LayerControl().add_to(fmap)
# lat/lon to map
# folium.LatLngPopup().add_to(fmap)
fmap.save('/var/www/html/pp.html')  # 存成 final.html

In [89]:
 times_micro

0       2019-12-20 00:24:13
1       2019-12-20 00:28:36
2       2019-12-20 00:27:54
3       2019-12-20 00:11:56
4       2019-12-20 00:26:31
5       2019-12-20 00:28:44
6       2019-12-19 16:37:00
7       2019-12-20 00:24:01
8       2019-12-20 00:27:25
9       2019-12-20 00:28:12
10      2019-12-20 00:26:37
11      2019-12-20 00:18:21
12      2019-12-19 18:01:25
13      2019-12-20 00:28:40
14      2019-12-20 00:27:54
15      2019-12-20 00:26:51
16      2019-12-19 16:11:47
17      2019-12-20 00:28:23
18      2019-12-20 00:28:40
19      2019-12-20 00:28:43
20      2019-12-20 00:27:11
21      2019-12-20 00:28:43
22      2019-12-20 00:10:58
23      2019-12-20 00:24:18
24      2019-12-20 00:22:31
25      2019-12-20 00:28:08
26      2019-12-19 15:37:35
27      2019-12-20 00:25:46
28      2019-12-20 00:25:05
29      2019-12-20 00:12:36
               ...         
2812    2019-12-20 00:17:11
2813    2019-12-20 00:17:10
2814    2019-12-20 00:16:41
2815    2019-12-20 00:17:05
2816    2019-12-20 0